In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
import logging
from sklearn.manifold import TSNE
import pandas as pd
import nltk
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt; plt.rcdefaults()
from sklearn.cluster import MeanShift, estimate_bandwidth
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.metrics.cluster import fowlkes_mallows_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt

from utils.utils import compute_gap, preprocessing, extract_tfidf_keywords, jaccard, FM, F1

In [2]:
StopWords = set(stopwords.words('english'))
porter = PorterStemmer()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Raw = [line.strip('\n') for line in open('/home/hamish/Documents/datasets/1.Raw Input.txt').readlines()]

True_cluster= pd.read_excel('/home/hamish/Documents/datasets/5.True Label (Ground Truth).xlsx',header=None)

TF02= pd.read_excel('/home/hamish/Documents/datasets/4.TF-IDF Filtered Version.xlsx')

In [3]:
nltk.download('punkt')
Cleaned_Corpus = [preprocessing(line) for line in Raw]
TF_Words = extract_tfidf_keywords(Cleaned_Corpus, 10)


[nltk_data] Downloading package punkt to /home/hamish/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
model = Word2Vec([word_tokenize(line) for line in Cleaned_Corpus], size=100, window=5, min_count=1, workers=15)
doc_vec = []
for f in TF_Words:
    doc_vec.append(np.mean(model[f], axis=0))
X = pd.DataFrame(doc_vec)
CBOW_model = Word2Vec(TF_Words, min_count=1,size=100)

2020-05-20 15:54:06,432 : INFO : collecting all words and their counts
2020-05-20 15:54:06,433 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-20 15:54:06,500 : INFO : collected 44910 word types from a corpus of 528135 raw words and 4770 sentences
2020-05-20 15:54:06,500 : INFO : Loading a fresh vocabulary
2020-05-20 15:54:06,623 : INFO : effective_min_count=1 retains 44910 unique words (100% of original 44910, drops 0)
2020-05-20 15:54:06,623 : INFO : effective_min_count=1 leaves 528135 word corpus (100% of original 528135, drops 0)
2020-05-20 15:54:06,730 : INFO : deleting the raw counts dictionary of 44910 items
2020-05-20 15:54:06,731 : INFO : sample=0.001 downsamples 12 most-common words
2020-05-20 15:54:06,731 : INFO : downsampling leaves estimated 525098 word corpus (99.4% of prior 528135)
2020-05-20 15:54:06,807 : INFO : estimated required memory for 44910 words and 100 dimensions: 58383000 bytes
2020-05-20 15:54:06,807 : INFO : resetting laye

In [5]:
db = DBSCAN(eps=0.1, min_samples=5).fit(X)

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [6]:
unique, counts = np.unique(labels, return_counts=True)
dict(zip(unique, counts))

objects = unique
y_pos = unique
performance =counts

plt.bar(objects, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('count')
plt.xlabel('clusters')
plt.title('DBSCAN clusters distribution')

for i in range(len(counts)):
    plt.annotate(str(performance[i]), xy=(objects[i],performance[i]))

plt.show()

{-1: 2303,
 0: 2283,
 1: 6,
 2: 7,
 3: 20,
 4: 13,
 5: 6,
 6: 11,
 7: 5,
 8: 8,
 9: 8,
 10: 10,
 11: 20,
 12: 11,
 13: 5,
 14: 5,
 15: 5,
 16: 7,
 17: 8,
 18: 4,
 19: 5,
 20: 6,
 21: 9,
 22: 5}